In [1]:
import numpy as np
import pandas as pd
import play_scraper

In [2]:
j=0
count=0

In [3]:
uname_arr=[]
date_arr=[]
score_arr=[]
url_arr=[]
appid_arr=[]
reviewid_arr=[]
reviewtitle_arr=[]
reviewtext_arr=[]
replydate_arr=[]
replytext_arr=[]
version_arr=[]
thumbsup_arr=[]

In [4]:
with open("test.txt", errors='ignore') as file:
    lines = file.readlines()
    

    
reviews=[]
no_of_apps = 0
no_of_revs = 0

    
    
for line in lines:
    splitted_line = line.split(",")
    if ':{"title":' in splitted_line[0]:
        no_of_apps+=1
        #print(" App_details",no_of_apps)
    elif ':[{"id":' in splitted_line[0]:# reviews processing
        no_of_revs+=1
        for i in range(len(splitted_line)):
            
            
            if '"userName":' in splitted_line[i] :
                count=count+1
                name = splitted_line[i].split(":")[1]
                uname_arr.append(name)#print(name, "sno", count)
                
            if '"date":' in splitted_line[i] :
                date = splitted_line[i].split(":")[1]
                date = date.split("T")[0]+'"'
                date_arr.append(date)##print(date)
                
            if '"scoreText":' in splitted_line[i] :
                score = splitted_line[i].split(":")[1]
                score_arr.append(score)#print(score)
                
            if '"title":' in splitted_line[i] :
                title = splitted_line[i].split(":")[1]
                reviewtitle_arr.append(title)#print(title)
                
            if '"text":' in splitted_line[i] :
                text = splitted_line[i].split(":")[1]
                j=i+1
                
                while(True):
                    
                    if '"replyDate":' in splitted_line[j]:
                        
                        i= j -1
                        break

                    elif '"replyDate":' not in splitted_line[j] :
                
                        text = text +" "+ splitted_line[j]
                        j = j+1
                    
                reviewtext_arr.append(text)#.append(text)#print(text)
            
            if '"replyDate":' in splitted_line[i] :
                reply_date = splitted_line[i].split(":")[1]
                reply_date = reply_date.split("T")[0]
                replydate_arr.append(reply_date)#uname.append(name)#print(reply_date)
        
            if '"replyText":' in splitted_line[i] :
                rep_text = splitted_line[i].split(":")[1]
                j=i+1

                while(True):
                    
                    if '"version":' in splitted_line[j] :
                        i= j -1
                        break

                    elif '"version":' not in splitted_line[j] :

                        rep_text = rep_text +" "+ splitted_line[j]
                        j = j+1

                replytext_arr.append(rep_text)#print("Reply Text", rep_text)
            

            if '"version":' in splitted_line[i] :
                version = splitted_line[i].split(":")[1]
                version_arr.append(version)#print(version)

            if '"thumbsUp":' in splitted_line[i] :
                thumbsup = splitted_line[i].split(":")[1]
                thumbsup_arr.append(thumbsup)#print("Thumbs up", thumbsup)

            if '"url":' in splitted_line[i] :
                fin_url="https:"
                ur = splitted_line[i].split(":")
                for ju in ur[2:]:
                    fin_url = fin_url + ju
                url_arr.append(fin_url)#print("Url", fin_url)

                app_id = fin_url.split("?")[1]
                app_id = app_id.split("&")[0]
                app_id = app_id.split("=")[1]
                appid_arr.append(app_id)#print("App Id", app_id)
                rev_id = fin_url.split("?")[1]
                rev_id = rev_id.split("&")
                rev_id = rev_id[1].split("=")[1]
                rev_id = rev_id[:len(rev_id)-1]
                reviewid_arr.append(rev_id)#print("Rev_id", rev_id)
        

In [5]:
unique_appid = []
s =set(appid_arr)
for each_s in s:
    unique_appid.append(each_s)

In [6]:
app_details_dictionary={}

In [7]:
for each_id in unique_appid:
    dets = []
    dictionary = play_scraper.details(each_id)
    dets.append(dictionary["title"])
    dets.append(dictionary["description"])
    dets.append(dictionary["category"])
    dets.append(dictionary["required_android_version"])
    dets.append(dictionary["size"])
    dets.append(dictionary["price"])
    dets.append(dictionary["reviews"])
    dets.append(dictionary["recent_changes"])
    dets.append(dictionary["installs"])
    dets.append(dictionary["content_rating"])
    app_details_dictionary[each_id]= dets

In [8]:
reviews = []
for i in range(len(reviewtext_arr)):
    review = []
    review.append(appid_arr[i])
    review.append(reviewid_arr[i])
    review.append(version_arr[i])
    review.append(date_arr[i])
    review.append(uname_arr[i])
    review.append(url_arr[i])
    review.append(score_arr[i])
    review.append(reviewtitle_arr[i])
    review.append(reviewtext_arr[i])
    review.append(thumbsup_arr[i])
    review.append(replydate_arr[i])
    review.append(replytext_arr[i])
    reviews.append(review)

In [9]:
final_list =[]

for r in range(len(reviews)):
    curr_appid = reviews[r][0]
    
    
    if curr_appid in app_details_dictionary:
        khali=[]
        khali = app_details_dictionary[curr_appid] + reviews[r]
        final_list.append(khali)

In [11]:
df = pd.DataFrame(final_list, columns=["AppName","AppDescription","Category", "AndroidVersionRequired", "Size", "Price","NoOfReviews","RecentChanges","Installs","ContentRating", "AppId", "ReviewId", "Version", "Date", "UserName", "Url", "Score", "RevTitle","Review","Thumbsup","Replydate","Replytext"])

In [12]:
df.to_csv("reviews.csv", index=False)